In [ ]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [ ]:
metric_view_yaml = f"""
CREATE OR REPLACE VIEW {catalog}.{schema}.orders_metric_view
WITH METRICS
LANGUAGE YAML
AS $$
    version: 1.1
    source: {catalog}.{schema}.indicators_metrics_fact
    joins:
      - name: geo
        source: {catalog}.{schema}.geo_dimension
        "on": source.geo_id = geo.id
      - name: magnitud
        source: {catalog}.{schema}.magnitud_dimension
        "on": source.magnitud_id = magnitud.id
      - name: tiempo
        source: {catalog}.{schema}.tiempo_dimension
        "on": source.tiempo_id = tiempo.id

    comment: "Metric view for analyzing the energy production indicators regarding energy generation, prices, etc."

    dimensions:
      - name: name
        expr: source.name
        comment: Name of the Indicator
        display_name: Indicator Name
        synonyms:
          - indicator
          - metric name
      - name: short_name
        expr: source.short_name
        comment: Short Name of the Indicator
        display_name: Indicator Short Name
        synonyms:
          - short name
          - metric short name
      - name: datetime_utc
        expr: source.datetime_utc
        comment: UTC datetime for the measurement
        display_name: Measurement Datetime (UTC)
        format:
          type: date_time
          date_format: year_month_day
          time_format: locale_hour_minute_second
          leading_zeros: false
        synonyms:
          - measurement time
          - utc datetime
      - name: magnitud
        expr: magnitud.name
        comment: "Name of the magnitud (e.g., type of measurement or indicator)"
        display_name: Magnitud Name
        synonyms:
          - magnitud
          - measurement type
          - indicator type
      - name: tiempo
        expr: tiempo.name
        comment: "Name of the tiempo (e.g., time period or granularity)"
        display_name: Tiempo Name
        synonyms:
          - tiempo
          - time period
          - granularity
      - name: geo
        expr: geo.name
        comment: "Name of the geo (e.g., geographic region or location)"
        display_name: Geo Name
        synonyms:
          - geo
          - region
          - location

    measures:
      - name: total_value
        expr: SUM(source.value)
        comment: Total Value
        display_name: Total Value
        format:
          type: number
          decimal_places:
            type: exact
            places: 2
          abbreviation: compact
        synonyms:
          - total value
          - total metric
$$
"""

In [ ]:
spark.sql(metric_view_yaml)